In [1]:
%matplotlib inline
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import pandas as pd
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)
import seaborn as sns
sns.set_style("whitegrid")
sns.set_context("poster")
from bs4 import BeautifulSoup
import requests
import json
import time

In [2]:
base_url = "https://www.racingchannel.com"
text = requests.get(base_url+"/results_archive.php").text
soup = BeautifulSoup(text, "html.parser")
table = soup.find("div", attrs={"class": "AccordionPanelContent"}).find_all("a")

In [3]:
locations = [url.get_text() for url in table] # get location text for each anchored element in table
# clean typos in original table
locations[1] = unicode("Arlington Park")
locations[34] = unicode("Indiana Downs")
loc_urls = [url.get("href") for url in table] # get url for each anchored element in table

In [6]:
%%time
date_dict = {}
for loc, loc_url in zip(locations, loc_urls):
    r1 = requests.get(base_url+loc_url).text
    s1 = BeautifulSoup(r1, "html.parser")
    cols = s1.find_all("td", attrs={"valign": "top"})
    links = []
    date_format = []
    dates = []
    for col in cols:
        months = col.find_all("a")
        for month in months:
            links.append(month.get("href"))
    for link in links:
        r2 = requests.get(base_url+link).text
        s2 = BeautifulSoup(r2, "html.parser")
        cal_urls = s2.find("table", attrs={"width": "567"}).find_all("a")
        for cal_url in cal_urls:
            dates.append(cal_url.get('href'))
        time.sleep(2)
    import re # http://regexr.com
    for item in dates:
        l = re.findall("(?:\d*\.)?\d+", item)
        date_format.append(l[0] + l[1] + l[2][4:])
    date_dict[loc] = date_format

CPU times: user 3min 39s, sys: 11.8 s, total: 3min 51s
Wall time: 4h 23min 23s


In [7]:
date_dict

{u'Aqueduct': [u'19981028',
  u'19981029',
  u'19981030',
  u'19981031',
  u'19981101',
  u'19981103',
  u'19981105',
  u'19981106',
  u'19981107',
  u'19981108',
  u'19981111',
  u'19981112',
  u'19981113',
  u'19981114',
  u'19981115',
  u'19981118',
  u'19981119',
  u'19981120',
  u'19981121',
  u'19981122',
  u'19981125',
  u'19981126',
  u'19981127',
  u'19981128',
  u'19981129',
  u'19981202',
  u'19981203',
  u'19981204',
  u'19981205',
  u'19981206',
  u'19981209',
  u'19981210',
  u'19981211',
  u'19981212',
  u'19981213',
  u'19981216',
  u'19981217',
  u'19981218',
  u'19981219',
  u'19981220',
  u'19981226',
  u'19981227',
  u'19981230',
  u'19990414',
  u'19990416',
  u'19990417',
  u'19990418',
  u'19990421',
  u'19990422',
  u'19990423',
  u'19990424',
  u'19990425',
  u'19990428',
  u'19990429',
  u'19990430',
  u'19990501',
  u'19990502',
  u'19990505',
  u'19990506',
  u'19990507',
  u'19990508',
  u'19990509',
  u'19991027',
  u'19991028',
  u'19991029',
  u'19991030

In [158]:
'''# http://stackoverflow.com/questions/7746832/scraping-and-parsing-google-search-results-using-python
from xgoogle.search import GoogleSearch, SearchError
for loc in locations:
    wt = random.uniform(2, 5)
    gs = GoogleSearch(loc)
    gs.results_per_page = 10
    results = gs.get_results()
    #Try not to annnoy Google, with a random short wait
    time.sleep(wt)'''

'# http://stackoverflow.com/questions/7746832/scraping-and-parsing-google-search-results-using-python\nfrom xgoogle.search import GoogleSearch, SearchError\nfor loc in locations:\n    wt = random.uniform(2, 5)\n    gs = GoogleSearch(loc)\n    gs.results_per_page = 10\n    results = gs.get_results()\n    #Try not to annnoy Google, with a random short wait\n    time.sleep(wt)'

In [100]:
# locations

In [70]:
zips = ['11420', '60005', 'NA', 'NA', 'NA', '94403', '11003', '43123', '33056', '00729', '55379', '25438',
        '40208', '23124', '70668', '92014', '19804', '42420', '98001', '70570', '70119', '62234', '91768', 
        '14425', '0', '94710', 'NA', 'NA', '49415', '33009', 'NA', '60804', '90305', '46013', '46176', '40510', 
        '42134', '20725', '71111', '75050', '90720', '07073', '07757', '26047', 'NA', '71901', '17028', 
        '19020', '21215', '97217', '50009', '73111', '78154', '45230', '77064', '91007', '12866', 'NA', 
        'NA', '60804', '02128', '33626', '44128', '21093', '85023', '41042', 'NA'] 

In [72]:
zips_dict = dict(zip(locations, zips))

In [4]:
# zips_dict

In [91]:
date = '20141225'
zip_code = '41042'
wunderground_url = 'http://api.wunderground.com/api/4a26cfc369eb7841/history_{}/q/{}.json'.format(date, zip_code)

In [185]:
json.loads(requests.get(wunderground_url).text)

{u'history': {u'dailysummary': [{u'coolingdegreedays': u'0',
    u'coolingdegreedaysnormal': u'0',
    u'date': {u'hour': u'12',
     u'mday': u'25',
     u'min': u'00',
     u'mon': u'12',
     u'pretty': u'12:00 PM EST on December 25, 2014',
     u'tzname': u'America/New_York',
     u'year': u'2014'},
    u'fog': u'0',
    u'gdegreedays': u'0',
    u'hail': u'0',
    u'heatingdegreedays': u'28',
    u'heatingdegreedaysnormal': u'33',
    u'humidity': u'',
    u'maxdewpti': u'34',
    u'maxdewptm': u'1',
    u'maxhumidity': u'82',
    u'maxpressurei': u'30.21',
    u'maxpressurem': u'1023',
    u'maxtempi': u'41',
    u'maxtempm': u'5',
    u'maxvisi': u'10',
    u'maxvism': u'16',
    u'maxwspdi': u'20',
    u'maxwspdm': u'32',
    u'meandewpti': u'30',
    u'meandewptm': u'-1',
    u'meanpressurei': u'30.04',
    u'meanpressurem': u'1017',
    u'meantempi': u'37',
    u'meantempm': u'3',
    u'meanvisi': u'10',
    u'meanvism': u'15',
    u'meanwdird': u'226',
    u'meanwdire': u'',

In [ ]:
weather_json = {}
for key in date_dict.keys():
    for fdate in date_dict.values():
        wunderground_url = 'http://api.wunderground.com/api/4a26cfc369eb7841/history_{}/q/{}.json'.format(fdate, zips_dict[key])
        weather_json[(key, fdate)] = json.loads(requests.get(wunderground_url).text)